In [3]:
import pandas as pd
import numpy as np

In [4]:
data = pd.read_csv('./data/weather_data_days.csv', sep=',')
data = data.drop(['text'], axis='columns')
data = data.drop(['day_end'], axis='columns')
data = data.drop(data[data['day_start'] < 1].index)
data = data.drop(data[data.iloc[:, 0:61].sum(axis = 1, skipna = True) == 0].index)

In [5]:
X= data.drop('day_start', axis=1)
y = data['day_start'].astype('int')

In [6]:
X.shape, y.shape

((11490, 61), (11490,))

In [7]:
X.head()

,4,30,29,31,13-14,21,25,7-8,12-13,18-19,...,2,28-29,14-15,27-28,13,6,8,19,16,3-4
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(random_state=17)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                      test_size=0.3, 
                                                      random_state=17)

In [10]:
X_train.shape, X_valid.shape

((8043, 61), (3447, 61))

In [11]:
from sklearn.model_selection import  cross_val_score
np.mean(cross_val_score(dtc, X_train, y_train, cv=5))

0.9829666150292663

In [12]:
%time
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(
    random_state=17,
    n_jobs=-1, max_features=2,
    )
rf.fit(X_train, y_train)

Wall time: 0 ns


RandomForestClassifier(max_features=2, n_jobs=-1, random_state=17)

In [13]:
%time
rf_valid_pred = rf.predict(X_valid)

Wall time: 0 ns


In [14]:
from sklearn.metrics import accuracy_score
accuracy_score(y_valid, rf_valid_pred)

0.9878154917319408

In [15]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import accuracy_score

In [16]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [17]:
rfc = RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True)

In [18]:
results = cross_val_score(rfc, X, y, cv=skf)

In [19]:
print("CV accuracy score: {:.2f}%".format(results.mean()*100))

CV accuracy score: 98.51%


In [20]:
# Инициализируем валидацию
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [21]:
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
trees_grid = range(20,30) 

In [22]:
# Обучаем на тренировочном датасете
for ntrees in trees_grid:
    rfc = RandomForestClassifier(n_estimators=ntrees, random_state=42, n_jobs=-1, oob_score=True)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)

c:\users\user\projects\pogoda_news_parser\venv\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\users\user\projects\pogoda_news_parser\venv\lib\site-packages\sklearn\ensemble\_forest.py:544: RuntimeWarning: invalid value encountered in true_divide
  decision = (predictions[k] /
c:\users\user\projects\pogoda_news_parser\venv\lib\site-packages\sklearn\ensemble\_forest.py:540: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
c:\users\user\projects\pogoda_news_parser\venv\lib\site-packages\sklearn\ensemble\_forest.py:544: RuntimeWarning: invalid value encountered in true_divide
  decision = (predictions[k] /
c:\users\user\projects\pogoda_news_parser\venv\lib\site-packages\sklearn\ens

In [23]:
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} trees".format(max(test_acc.mean(axis=1))*100, 
                                                        trees_grid[np.argmax(test_acc.mean(axis=1))]))

Best accuracy on CV is 98.55% with 28 trees


In [24]:
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
max_depth_grid = range(10,35)

In [25]:
# Обучаем на тренировочном датасете
for max_depth in max_depth_grid:
    rfc = RandomForestClassifier(n_estimators=51, random_state=42, n_jobs=-1, oob_score=True, max_depth=max_depth)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} max_depth".format(max(test_acc.mean(axis=1))*100, 
                                                        max_depth_grid[np.argmax(test_acc.mean(axis=1))]))

Best accuracy on CV is 98.18% with 34 max_depth


In [26]:
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
min_samples_leaf_grid = [1, 3, 5, 7, 9, 11, 13, 15, 17, 20, 22, 24]

In [27]:
# Обучаем на тренировочном датасете
for min_samples_leaf in min_samples_leaf_grid:
    rfc = RandomForestClassifier(n_estimators=51, random_state=17, n_jobs=-1, 
                                 oob_score=True, min_samples_leaf=min_samples_leaf, max_depth=34)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} min_samples_leaf".format(max(test_acc.mean(axis=1))*100, 
                                                                min_samples_leaf_grid[np.argmax(test_acc.mean(axis=1))]))

Best accuracy on CV is 98.15% with 1 min_samples_leaf


In [28]:
# Создаем списки для сохранения точности на тренировочном и тестовом датасете
train_acc = []
test_acc = []
temp_train_acc = []
temp_test_acc = []
max_features_grid = [2, 4, 6, 8, 10, 12, 14, 16]

# Обучаем на тренировочном датасете
for max_features in max_features_grid:
    rfc = RandomForestClassifier(n_estimators=51, random_state=17, n_jobs=-1, 
                                 oob_score=True, max_depth=34, max_features= 2)
    temp_train_acc = []
    temp_test_acc = []
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]
        rfc.fit(X_train, y_train)
        temp_train_acc.append(rfc.score(X_train, y_train))
        temp_test_acc.append(rfc.score(X_test, y_test))
    train_acc.append(temp_train_acc)
    test_acc.append(temp_test_acc)
    
train_acc, test_acc = np.asarray(train_acc), np.asarray(test_acc)
print("Best accuracy on CV is {:.2f}% with {} max_features".format(max(test_acc.mean(axis=1))*100, 
                                                        max_features_grid[np.argmax(test_acc.mean(axis=1))]))

Best accuracy on CV is 98.54% with 2 max_features


In [35]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train = y.iloc[train_index]
    y_test = y.iloc[test_index]
rfc = RandomForestClassifier(n_estimators=28, random_state=17, n_jobs=-1, 
                                 oob_score=True, max_depth=34, max_features= 2)

In [36]:
X_train.shape,  X_test.shape

((9192, 61), (2298, 61))

In [37]:
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=34, max_features=2, n_estimators=28, n_jobs=-1,
                       oob_score=True, random_state=17)

In [38]:
accuracy_score(y_test, rfc.predict(X_test))

0.9882506527415144

In [39]:
accuracy_score(y_test, rfc.predict(X_test))

0.9882506527415144